The following is an implementation of the 2001 academic paper authored by Joseph D. Piotroski titled,'Value Investing: The Use of Historical Financial Statement Information to Separate Winners from Losers' published in the 'Journal of Accounting Research'.

You may input the ticker symbols of the companies you're interested in, and their F-score will be generated as output. (API works for U.S. stocks)

In [26]:
import pandas as pd
import requests
import json

def statement(ticker,statement,period):  #function to get the required report of the concerned company
    if statement =='IS':
        temp='income-statement'
    elif statement =='BS':
        temp='balance-sheet-statement'
    elif statement =='CFR':
        temp='ratios'
    else:
        temp='cash-flow-statement'
        
    if period=='Q':
        temp2 = 'period=quarter'
    else:
        temp2 = ''
    bs = requests.get(f'https://financialmodelingprep.com/api/v3/'+temp+'/'+ticker+'?'+temp2+'&apikey=')#Enter your API key here.
    #Getting an API key is free on the above website. Just register with your email, and input your personal key here.
    bs = bs.json()
    bs = pd.DataFrame.from_dict(bs) #Manipulation of data for better accessibility
    bs = bs.T
    bs.columns = bs.iloc[0]
    bs = bs.iloc[1:]
    bs = bs.iloc[:,:4]
    #bs 
    return bs
#statement('AAPL','IS','A') #Just for demonstration

date,2019-09-28,2018-09-29,2017-09-30,2016-09-24
symbol,AAPL,AAPL,AAPL,AAPL
fillingDate,2019-10-31,2018-11-05,2017-11-03,2016-10-26
acceptedDate,2019-10-30 18:12:36,2018-11-05 08:01:40,2017-11-03 08:01:37,2016-10-26 16:42:16
period,FY,FY,FY,FY
revenue,260174000000,265595000000,229234000000,215639000000
costOfRevenue,161782000000,163756000000,141048000000,131376000000
grossProfit,98392000000,101839000000,88186000000,84263000000
grossProfitRatio,0.378178,0.383437,0.384699,0.39076
researchAndDevelopmentExpenses,16217000000,14236000000,11581000000,10045000000
generalAndAdministrativeExpenses,18245000000,16705000000,15261000000,14194000000


This is how the Income Statement(last 4 years) of Apple looks like, after the manipulation.

In [27]:
#The following can be clustered together to reduce API requests, but this provides more readability/clarity of operation.
def net_income(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    return float(df.loc['netIncome'][t])

def roa(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    boy_assets=float(df.loc['totalAssets'][t+1])
    return net_income(ticker,t)/boy_assets

def delroa(ticker,t=0):
    return roa(ticker,t)- roa(ticker,t+1)
    
def cfo(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'CF','A'))
    return float(df.loc['operatingCashFlow'][t])

def ltdebt(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['longTermDebt'][t]),float(df.loc['longTermDebt'][t+1])
    df2=pd.DataFrame(statement(ticker,'BS','A'))
    a,b,c=float(df2.loc['totalAssets'][t]),float(df2.loc['totalAssets'][t+1]),float(df2.loc['totalAssets'][t+2])
    av_assets1=(a+b)/2
    av_assets2=(b+c)/2
    return q/av_assets2 -p/av_assets1

def current_ratio(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['totalCurrentAssets'][t]),float(df.loc['totalCurrentAssets'][t+1])
    r,s=float(df.loc['totalCurrentLiabilities'][t]),float(df.loc['totalCurrentLiabilities'][t+1])
    cr_1,cr_2=p/r,q/s
    return cr_1-cr_2

def equity(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['retainedEarnings'][0]),float(df.loc['retainedEarnings'][1])
    r,s=float(df.loc['totalStockholdersEquity'][0]),float(df.loc['totalStockholdersEquity'][1])
    return (s-q)-(r-p)

def gross_margin(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    p,q=float(df.loc['grossProfitRatio'][t]),float(df.loc['grossProfitRatio'][t+1])
    return p-q

def Asset_Turnover_Ratio(ticker,t=0):
    df1=pd.DataFrame(statement(ticker,'IS','A'))
    df2=pd.DataFrame(statement(ticker,'BS','A'))
    a1,a2=float(df2.loc['totalAssets'][t]),float(df2.loc['totalAssets'][t+1])
    atr1=float(df1.loc['revenue'][0])/a1
    atr2=float(df1.loc['revenue'][1])/a2
    return atr1-atr2

def Piotroski_Score(ticker,t=0):
    df=pd.DataFrame(statement(ticker,'CFR','A'))  #Price to book ratio (this strategy is for P/B<1)
    PBR= df.loc['priceToBookRatio'][t]
    Pscore=0
    if roa(ticker,t)>0: #Return on Assets   CALCULATION OF F SCORE
        Pscore+=1
    if delroa(ticker,t)>0: #Change in ROA
        Pscore+=1
    if cfo(ticker,t)>0: #Cash flow from operations
        Pscore+=1
    if cfo(ticker,t)>net_income(ticker): #Accruals
        Pscore+=1
    if ltdebt(ticker,t)>0: #Delta Leverage
        Pscore+=1
    if current_ratio(ticker,t)>0: #Change in C.R.(Liquidity)
        Pscore+=1
    if equity(ticker,t)>0: #Equity raise
        Pscore+=1
    if gross_margin(ticker,t)>0: #Increase in gross margin
        Pscore+=1
    if Asset_Turnover_Ratio(ticker,t)>0: #
        Pscore+=1
    return Pscore,PBR

In [28]:
tickers=['AAPL','GOOG','TSLA','F','MSFT'] #Remember, piotroski value investing strategy is only meant for low PBratio firms(<1)
#Enter the tickers you're interested in.
data={}
for ticker in tickers:
    data[ticker]= [Piotroski_Score(ticker)]
    final=pd.DataFrame(data2,index=[0])  
final #(FScore,P/B ratio)

,0
AAPL,"(5, 12.709658271815046)"
GOOG,"(4, 2.5053945318255377)"
TSLA,"(5, 17.71958748866727)"
F,"(4, 1.0580828687660089)"
MSFT,"(7, 10.52384979966774)"


Lots of calls involved, each taking its own time. Takes about a minute to furnish the results.

Note:This strategy is meant for financially distressed firms, which are undervalued by the market. Hence we first need to find a list of value stocks. For that, get a list of all the tickers being traded, and select only those with P/B<1, using the function included in Piotroski_score(), seperately. Due to API request restrictions, I couldn't incorporate the same. The strategy can be used on NSE/BSE stocks as well.

Read Piotroski's Paper on value investing for the actual strategy.